<a href="https://colab.research.google.com/github/danilopinho75/Portfolio-Analise-de-Dados-Pizzaria/blob/ingestao-de-dados/ingestao_de_dados_portfolio_pizzaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importações de bibliotecas

In [ ]:
!pip install jupysql --quiet

In [ ]:
%load_ext sql

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
import sqlite3

In [ ]:
# Configuração para jupysql
%config SqlMagic.named_parameters = "enabled"
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

### Inserção de dados

In [ ]:
# Configuração dos dados para as pizzas
pizzas = [
    {"name": "Calabresa e Queijo", "category": "Tradicional", "ingredients": "Calabresa, Queijo, Orégano", "unit_price": 35.0},
    {"name": "Marguerita", "category": "Especial", "ingredients": "Queijo, Tomate, Manjericão", "unit_price": 37.0},
    {"name": "Queijo", "category": "Tradicional", "ingredients": "Mussarela, Orégano", "unit_price": 30.0},
    {"name": "Milho e Bacon", "category": "Especial", "ingredients": "Milho, Bacon, Mussarela", "unit_price": 40.0},
    {"name": "Portuguesa", "category": "Tradicional", "ingredients": "Mussarela, Presunto, Ovo, Cebola, Azeitona", "unit_price": 42.0},
    {"name": "Frango com Catupiry", "category": "Especial", "ingredients": "Frango, Catupiry, Orégano", "unit_price": 45.0},
    {"name": "4 Queijos", "category": "Especial", "ingredients": "Mussarela, Provolone, Parmesão, Gorgonzola", "unit_price": 50.0},
    {"name": "Pepperoni", "category": "Tradicional", "ingredients": "Mussarela, Pepperoni, Orégano", "unit_price": 40.0},
    {"name": "Strogonoff de Carne", "category": "Especial", "ingredients": "Carne, Creme, Batata Palha", "unit_price": 47.0},
    {"name": "Strogonoff de Frango", "category": "Especial", "ingredients": "Frango, Creme, Batata Palha", "unit_price": 47.0},
    {"name": "Moda da Casa", "category": "Especial", "ingredients": "Mussarela, Presunto, Azeitona, Calabresa, Milho", "unit_price": 50.0},
    {"name": "Calabresa Acebolada", "category": "Tradicional", "ingredients": "Calabresa, Cebola, Mussarela", "unit_price": 38.0},
    {"name": "Frango com Cheddar", "category": "Especial", "ingredients": "Frango, Cheddar, Orégano", "unit_price": 45.0},
    {"name": "Frango com Bacon", "category": "Especial", "ingredients": "Frango, Bacon, Queijo, Orégano", "unit_price": 46.0}
]

sizes = ["Pequena", "Média", "Grande"]
delivery_methods = ["Delivery", "Retirada"]
payment_methods = ["Cartão de crédito", "Dinheiro", "PIX", "Vale-refeição"]
promotions = ["Combo Família", "Desconto PIX", "Segunda em Dobro", None]
cities_states = [("São Paulo", "SP"), ("Rio de Janeiro", "RJ"), ("Curitiba", "PR"), ("Belo Horizonte", "MG"), ("Florianópolis", "SC"), ("Maringá", "PR")]

# Parâmetros de data e geração dos dados
start_date = datetime(2021, 11, 1)
end_date = datetime(2024, 9, 30)
data_list = []
order_id = 1
current_date = start_date

while current_date <= end_date:
    # Define um número de pedidos variado por ano
    if current_date.year == 2021:
        num_orders = random.randint(5, 20)
    elif current_date.year == 2022:
        num_orders = random.randint(15, 30)
    elif current_date.year == 2023:
        num_orders = random.randint(25, 40)
    else:
        num_orders = random.randint(35, 50)

    # Geração dos pedidos para o dia atual
    for _ in range(num_orders):
        pizza = random.choice(pizzas)
        quantity = random.randint(1, 5)
        pizza_size = random.choice(sizes)
        delivery_method = random.choice(delivery_methods)
        payment_method = random.choice(payment_methods)
        promotion_applied = random.choice([0, 1])
        name_promotion = random.choice(promotions) if promotion_applied else None
        city, state = random.choice(cities_states)

        total_price = pizza["unit_price"] * quantity
        order_time = (datetime.min + timedelta(minutes=random.randint(18 * 60, 23 * 60))).time()

        # Adiciona os dados ao `data_list`
        data_list.append([
            order_id, pizza["name"], quantity, current_date.strftime('%Y-%m-%d'),
            order_time.strftime('%H:%M:%S'), pizza["unit_price"], total_price,
            pizza_size, pizza["category"], pizza["ingredients"],
            delivery_method, payment_method, promotion_applied,
            name_promotion, city, state
        ])
        order_id += 1
    current_date += timedelta(days=1)

# Cria um DataFrame a partir dos dados gerados
df = pd.DataFrame(data_list, columns=[
    "order_id", "pizza_name", "quantity", "order_date", "order_time", "unit_price",
    "total_price", "pizza_size", "pizza_category", "pizza_ingredients",
    "delivery_method", "payment_method", "promotion_applied",
    "name_promotion", "customer_city", "customer_state"
])

# Conexão com o SQLite e exportação para a tabela
conn = sqlite3.connect("pizzaria.db")
df.to_sql("pedidos_pizzaria", conn, if_exists="replace", index=False)

print("Dados inseridos com sucesso!")
conn.close()


Dados inseridos com sucesso!


In [ ]:
df.head()

,order_id,pizza_name,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,delivery_method,payment_method,promotion_applied,name_promotion,customer_city,customer_state
0,1,Queijo,3,2021-11-01,20:32:00,30.0,90.0,Pequena,Tradicional,"Mussarela, Orégano",Retirada,PIX,0,None,Florianópolis,SC
1,2,Frango com Bacon,2,2021-11-01,20:39:00,46.0,92.0,Grande,Especial,"Frango, Bacon, Queijo, Orégano",Retirada,Dinheiro,1,Combo Família,Maringá,PR
2,3,Strogonoff de Carne,4,2021-11-01,22:00:00,47.0,188.0,Grande,Especial,"Carne, Creme, Batata Palha",Retirada,Vale-refeição,0,None,São Paulo,SP
3,4,Calabresa Acebolada,2,2021-11-01,18:20:00,38.0,76.0,Pequena,Tradicional,"Calabresa, Cebola, Mussarela",Delivery,Cartão de crédito,1,Desconto PIX,Maringá,PR
4,5,Strogonoff de Carne,4,2021-11-01,19:53:00,47.0,188.0,Média,Especial,"Carne, Creme, Batata Palha",Delivery,Dinheiro,0,None,Curitiba,PR


### Conectando ao banco de dados

In [ ]:
%sql sqlite:///pizzaria.db

Connecting to 'sqlite:///pizzaria.db'

### Consultas SQL

In [ ]:
%%sql

SELECT * FROM pedidos_pizzaria LIMIT 5;

Running query in 'sqlite:///pizzaria.db'

order_id,pizza_name,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,delivery_method,payment_method,promotion_applied,name_promotion,customer_city,customer_state
1,Queijo,3,2021-11-01,20:32:00,30.0,90.0,Pequena,Tradicional,"Mussarela, Orégano",Retirada,PIX,0,None,Florianópolis,SC
2,Frango com Bacon,2,2021-11-01,20:39:00,46.0,92.0,Grande,Especial,"Frango, Bacon, Queijo, Orégano",Retirada,Dinheiro,1,Combo Família,Maringá,PR
3,Strogonoff de Carne,4,2021-11-01,22:00:00,47.0,188.0,Grande,Especial,"Carne, Creme, Batata Palha",Retirada,Vale-refeição,0,None,São Paulo,SP
4,Calabresa Acebolada,2,2021-11-01,18:20:00,38.0,76.0,Pequena,Tradicional,"Calabresa, Cebola, Mussarela",Delivery,Cartão de crédito,1,Desconto PIX,Maringá,PR
5,Strogonoff de Carne,4,2021-11-01,19:53:00,47.0,188.0,Média,Especial,"Carne, Creme, Batata Palha",Delivery,Dinheiro,0,None,Curitiba,PR
